In [ ]:
import random

random.seed(123)

## IC Performance

In [ ]:
import numpy as np
import pandas

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
Le = 1.6858
L_SSGR =  1
# df = 5
df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

total_runs = []
high_run_lengths = []
low_run_lengths = []
previous_high_run_length = None
previous_low_run_length = None
for i in range(sim):
    zi_1 = exp_w  # Phase I mean as starting EWMA value
    # xi = np.random.normal(0, 1, m)  # Phase I data
    # xi = np.random.standard_t(df, m)  # Phase I data t
    xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
    xi=(xi-shape_param)/np.sqrt(shape_param)
    high_count = 0
    low_count = 0
    signaled = False
    total_run = 0
    while not signaled:
        high_count += 1
        low_count += 1
        total_run += 1
        # yi = np.random.normal(0, 1, n)  # Phase II data
        # yi = np.random.standard_t(df, n)  # Phase II data t
        yi = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
        yi=(yi-shape_param)/np.sqrt(shape_param)
        comb = np.concatenate((xi, yi))
        ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
        W = ranks[-n:].sum()  # Sum of ranks for new sample

        # Calculate W-EWMA
        zi = lambda_ * W + (1 - lambda_) * zi_1
        zi_1 = zi
        # Check control limits and evaluate run length
        if zi >= UCL:
            if previous_high_run_length is None:
                # Check if the first nonconforming group is below the threshold
                if high_count <= L_SSGR:
                    high_run_lengths.append(high_count)
                    total_runs.append(total_run)
                    signaled = True
            elif previous_high_run_length <= L_SSGR and high_count <= L_SSGR:
                high_run_lengths.append(high_count)
                total_runs.append(total_run)
                signaled = True

            previous_high_run_length = high_count
            high_count = 0  # Reset count after nonconforming group and no signal

        if zi <= LCL:
            if previous_low_run_length is None:
                # Check if the first nonconforming group is below the threshold
                if low_count <= L_SSGR:
                    low_run_lengths.append(low_count)
                    total_runs.append(total_run)
                    signaled = True
            elif previous_low_run_length <= L_SSGR and low_count <= L_SSGR:
                low_run_lengths.append(low_count)
                total_runs.append(total_run)
                signaled = True

            previous_low_run_length = low_count
            low_count = 0  # Reset count after nonconforming group and no signal

run_lengths = np.array(total_runs)
print(f"Mean Run Length: {run_lengths.mean()}")
print(f"Standard Deviation: {run_lengths.std()}")
print(f"5th Percentile: {np.percentile(run_lengths, 5)}")
print(f"95th Percentile: {np.percentile(run_lengths, 95)}")
print(f"Median: {np.percentile(run_lengths, 50)}")


## With Shifts

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
np.random.seed(123)
# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.05
Le = 2.195
L_SSGR =  1
# df = 5
df = 20
shape_param = 1  # Shape parameter for the gamma distribution
# shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution
# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

# Simulating Phase I and Phase II

# Delta values
deltas = np.arange(0, 3.25, 0.25)

# DataFrame to store results
results = pd.DataFrame(columns=['Delta', 'ARL', 'SDRL', 'MDRL'])
for delta in deltas:
    print(delta)
    high_run_lengths = []
    low_run_lengths = []
    previous_high_run_length = None
    previous_low_run_length = None
    total_runs = []
    for i in tqdm(range(sim)):
        zi_1 = exp_w  # Phase I mean as starting EWMA value
        # xi = np.random.normal(0, 1, m)  # Phase I data Normal
        # xi = np.random.standard_t(df, m)  # Phase I data t
        xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
        xi=(xi-shape_param)/np.sqrt(shape_param)
        high_count = 0
        low_count = 0
        signaled = False
        total_run = 0
        while not signaled:
            high_count += 1
            low_count += 1
            total_run += 1
            # yi = np.random.normal(delta, 1, n)  # Phase II data
            # yi = np.random.standard_t(df, n) + delta  # Phase II data t
            y1 = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
            yi = (y1 - shape_param) / np.sqrt(shape_param) + (delta)
            comb = np.concatenate((xi, yi))
            ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
            W = ranks[-n:].sum()  # Sum of ranks for new sample

            # Calculate W-EWMA
            zi = lambda_ * W + (1 - lambda_) * zi_1
            zi_1 = zi
            # Check control limits and evaluate run length
            if zi >= UCL:
                if previous_high_run_length is None:
                    # Check if the first nonconforming group is below the threshold
                    if high_count <= L_SSGR:
                        high_run_lengths.append(high_count)
                        total_runs.append(total_run)
                        signaled = True
                elif previous_high_run_length <= L_SSGR and high_count <= L_SSGR:
                    high_run_lengths.append(high_count)
                    total_runs.append(total_run)
                    signaled = True

                previous_high_run_length = high_count
                high_count = 0  # Reset count after nonconforming group and no signal

            if zi <= LCL:
                if previous_low_run_length is None:
                    # Check if the first nonconforming group is below the threshold
                    if low_count <= L_SSGR:
                        low_run_lengths.append(low_count)
                        total_runs.append(total_run)
                        signaled = True
                elif previous_low_run_length <= L_SSGR and low_count <= L_SSGR:
                    low_run_lengths.append(low_count)
                    total_runs.append(total_run)
                    signaled = True

                previous_low_run_length = low_count
                low_count = 0  # Reset count after nonconforming group and no signal

    run_lengths = np.array(total_runs)
    ARL = run_lengths.mean()
    SDRL = run_lengths.std()
    MDRL = np.percentile(run_lengths, 50)

    # Append to results
    results_row = pd.DataFrame({'Delta': [delta], 'ARL': [ARL], 'SDRL': [SDRL], 'MDRL': [MDRL]})
    results = pd.concat([results, results_row], ignore_index=True)

results


0.0


100%|██████████| 10000/10000 [01:35<00:00, 105.24it/s]


0.25


100%|██████████| 10000/10000 [00:07<00:00, 1302.31it/s]


0.5


...

2.75


100%|██████████| 10000/10000 [00:01<00:00, 5271.79it/s]


3.0


100%|██████████| 10000/10000 [00:01<00:00, 5227.00it/s]


,Delta,ARL,SDRL,MDRL
0,0.00,209.5299,304.027319,99.0
1,0.25,18.8127,49.777085,13.0
2,0.50,8.1562,1.952281,8.0
3,0.75,6.4115,0.855668,6.0
4,1.00,5.6614,0.595777,6.0
5,1.25,5.1942,0.400857,5.0
6,1.50,5.0318,0.177732,5.0
7,1.75,5.0008,0.059995,5.0
8,2.00,4.9887,0.105699,5.0
9,2.25,4.9559,0.205804,5.0


# Multiple Comparison

In [7]:
import numpy as numpy
import pandas as pandas
from tqdm import tqdm

numpy.random.seed(123)

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = numpy.sqrt(n * m * (n + m + 1) / 12)
L_SSGR = 3
# df = 5
# df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

# Delta values
deltas = numpy.arange(0, 3.25, 0.25)

# Parameters for lambda and Le
lambda_values = [0.05, 0.5]
Le_values = [2.2199, 1.84]

# DataFrame to store results
results = pandas.DataFrame(columns=['Delta', 'ARL (0.05)', 'SDRL (0.05)', 'MDRL (0.05)', 'ARL (0.5)', 'SDRL (0.5)', 'MDRL (0.5)'])

for lambda_, Le in zip(lambda_values, Le_values):
    # Control Limits
    LCL = exp_w - Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    UCL = exp_w + Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    
    arl_column = f'ARL ({lambda_})'
    sdrl_column = f'SDRL ({lambda_})'
    mdrl_column = f'MDRL ({lambda_})'
    
    for delta in deltas:
        print(f"Delta: {delta}, Lambda: {lambda_}")
        high_run_lengths = []
        low_run_lengths = []
        previous_high_run_length = None
        previous_low_run_length = None
        total_runs = []
        
        for i in tqdm(range(sim)):
            zi_1 = exp_w  # Phase I mean as starting EWMA value
            
            # Phase I data (Gamma distribution standardized to zero mean and unit variance)
            # xi = numpy.random.normal(0, 1, m)  # Phase I data Normal
            # xi = numpy.random.standard_t(df, m)  # Phase I data t
            xi = numpy.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
            xi = (xi - shape_param) / numpy.sqrt(shape_param)
            
            high_count = 0
            low_count = 0
            signaled = False
            total_run = 0
            
            while not signaled:
                high_count += 1
                low_count += 1
                total_run += 1
                
                # Phase II data (Gamma distribution standardized and shifted by delta)
                # yi = numpy.random.normal(delta, 1, n)  # Phase II data
                # yi = numpy.random.standard_t(df, n) + delta  # Phase II data t
                y1 = numpy.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
                yi = (y1 - shape_param) / numpy.sqrt(shape_param) + delta
                
                comb = numpy.concatenate((xi, yi))
                ranks = numpy.argsort(numpy.argsort(comb)) + 1  # Rank calculation
                W = ranks[-n:].sum()  # Sum of ranks for new sample
    
                # Calculate W-EWMA
                zi = lambda_ * W + (1 - lambda_) * zi_1
                zi_1 = zi
                
                # Check control limits and evaluate run length
                if zi >= UCL:
                    if previous_high_run_length is None:
                        # Check if the first nonconforming group is below the threshold
                        if high_count <= L_SSGR:
                            high_run_lengths.append(high_count)
                            total_runs.append(total_run)
                            signaled = True
                    elif previous_high_run_length <= L_SSGR and high_count <= L_SSGR:
                        high_run_lengths.append(high_count)
                        total_runs.append(total_run)
                        signaled = True
    
                    previous_high_run_length = high_count
                    high_count = 0  # Reset count after nonconforming group and no signal
    
                if zi <= LCL:
                    if previous_low_run_length is None:
                        # Check if the first nonconforming group is below the threshold
                        if low_count <= L_SSGR:
                            low_run_lengths.append(low_count)
                            total_runs.append(total_run)
                            signaled = True
                    elif previous_low_run_length <= L_SSGR and low_count <= L_SSGR:
                        low_run_lengths.append(low_count)
                        total_runs.append(total_run)
                        signaled = True
    
                    previous_low_run_length = low_count
                    low_count = 0  # Reset count after nonconforming group and no signal
    
        run_lengths = numpy.array(total_runs)
        ARL = run_lengths.mean()
        SDRL = run_lengths.std()
        MDRL = numpy.percentile(run_lengths, 50)
        
        # Store results
        if delta in results['Delta'].values:
            results.loc[results['Delta'] == delta, arl_column] = ARL
            results.loc[results['Delta'] == delta, sdrl_column] = SDRL
            results.loc[results['Delta'] == delta, mdrl_column] = MDRL
        else:
            new_row = pandas.DataFrame({'Delta': [delta], arl_column: [ARL], sdrl_column: [SDRL], mdrl_column: [MDRL]})
            results = pandas.concat([results, new_row], ignore_index=True)

# Format the columns as (ARL_0.05, ARL_0.5)
results['ARL'] = results.apply(lambda row: f"({row['ARL (0.05)']:.2f}, {row['ARL (0.5)']:.2f})", axis=1)
results = results[['Delta', 'ARL']]


results


Delta: 0.0, Lambda: 0.05


100%|██████████| 10000/10000 [01:18<00:00, 126.73it/s]


Delta: 0.25, Lambda: 0.05


100%|██████████| 10000/10000 [00:14<00:00, 677.66it/s]


Delta: 0.5, Lambda: 0.05


...

Delta: 2.75, Lambda: 0.5


100%|██████████| 10000/10000 [00:00<00:00, 21774.53it/s]


Delta: 3.0, Lambda: 0.5


100%|██████████| 10000/10000 [00:00<00:00, 22014.03it/s]


,Delta,ARL
0,0.00,"(198.28, 198.10)"
1,0.25,"(42.69, 58.32)"
2,0.50,"(11.66, 7.90)"
3,0.75,"(7.84, 2.91)"
4,1.00,"(6.18, 1.76)"
5,1.25,"(4.89, 1.36)"
6,1.50,"(3.76, 1.15)"
7,1.75,"(3.17, 1.05)"
8,2.00,"(3.00, 1.01)"
9,2.25,"(2.96, 1.00)"
